In [8]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy

In [4]:
x = [[0,0], [1,0], [0,1], [1,1]]
y = [0,1,1,0]

x = np.asarray(x)
y = np.asarray(y)
y = to_categorical(y)

x, y

(array([[0, 0],
        [1, 0],
        [0, 1],
        [1, 1]]), array([[1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.]], dtype=float32))

In [5]:
x_in = Input(shape=(2,))
s = Dense(5, activation='relu')(x_in)
s = Dense(5, activation='relu')(s)
s = Dense(2, activation='softmax')(s)

model = Model(inputs=x_in, outputs=s)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 5)                 15        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 12        
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [12]:
y_pred = model.predict(x)
y_pred

array([[0.5       , 0.5       ],
       [0.49469516, 0.5053049 ],
       [0.4651944 , 0.53480554],
       [0.49271408, 0.50728595]], dtype=float32)

In [11]:
loss = categorical_crossentropy(y, y_pred)
loss.numpy()

array([0.6931472 , 0.68259335, 0.625852  , 0.70782626], dtype=float32)